In [1]:
import numpy as np
import torch
from editings import styleclip_global_utils
from utils.models_utils import load_generators, load_old_G, load_from_pkl_model
from utils.edit_utils import get_affine_layers, load_stylespace_std, to_styles

In [2]:
gen = load_from_pkl_model( load_old_G() )

In [3]:
affine_layers = get_affine_layers(gen.synthesis)

In [4]:
neutral_class = 'face'
beta = 0.1
edit_names = ['happy', 'angry']

In [5]:
direction_happy = styleclip_global_utils.get_direction(neutral_class, 'happy', beta)
direction_angry = styleclip_global_utils.get_direction(neutral_class, 'angry', beta)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.96it/s]


In [6]:
edit_happy = to_styles(direction_happy, affine_layers)
edit_angry = to_styles(direction_angry, affine_layers)

In [8]:
for i in range(len(edit_happy)):
    happy_len = torch.norm(edit_happy[i], 2)
    angry_len = torch.norm(edit_angry[i], 2)
    if happy_len > 1e-4 and angry_len > 1e-4:
        edit_happy[i] /= happy_len
        edit_angry[i] /= angry_len
        print(torch.dot(edit_happy[i], edit_angry[i]).item())

0.0
0.0
0.054443359375
-0.08465576171875
0.0
0.0
0.0
0.0


In [10]:
styleclip_global_utils.get_direction(neutral_class, 'anger', 0.1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.99it/s]


tensor([ 0.0000, -0.0000,  0.0000,  ..., -0.5640,  0.0000, -0.0000],
       device='cuda:0', dtype=torch.float16)

In [11]:
pivots = np.load('w_vectors/e4e_yukun_happy_anger.npy')
from configs import global_config
pivots = torch.Tensor(pivots).to(global_config.device)

In [12]:
from utils.edit_utils import w_to_styles
styles = w_to_styles(pivots, affine_layers)

In [13]:
styles_origin = w_to_styles(pivots[0][None], affine_layers)

In [15]:
for i in range(len(edit_happy)):
    edit_happy[i] = edit_happy[i].float()
    edit_angry[i] = edit_angry[i].float()

In [17]:
happy_components = []
angry_components = []
for i in range(len(styles)):
    happy_components.append( torch.matmul((styles[i] - styles_origin[i]), edit_happy[i]) )
    angry_components.append( torch.matmul((styles[i] - styles_origin[i]), edit_angry[i]) )

In [19]:
happy_comp_tensor = torch.vstack(happy_components)
angry_comp_tensor = torch.vstack(angry_components)
print(happy_comp_tensor.shape)
print(angry_comp_tensor.shape)

torch.Size([26, 136])
torch.Size([26, 136])


In [26]:
happy_comp_np = happy_comp_tensor.cpu().numpy()
angry_comp_np = angry_comp_tensor.cpu().numpy()
print(happy_comp_np.shape)
print(angry_comp_np.shape)
np.save('TEST/happy_comp.npy', happy_comp_np)
np.save('TEST/angry_comp.npy', angry_comp_np)

(26, 136)
(26, 136)
